# Meetup Project


What is in a Meetup API request?

1. https://api.meetup.com: The API host.
2. /find/groups: The method
3. zip=11211&radius=1&category=25: The args. Here is where you may add the API key.
    From what I can tell, the only args you can give are zip, radius, and category. So maybe you need to go through the zips 
4. &order=members:  How to order the results.
 
* To find last event in the list: db.events.find().limit(1).sort({$natural:-1})

ssh -L localhost:8888:localhost:8888 -i "ecdoso.pem" ec2-user@ec2-52-14-250-184.us-east-2.compute.amazonaws.com


# Imports / Pymongo Setup

In [21]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import requests
import json
import warnings
import time
from pprint import pprint
import sys
sys.path.append('./src')
warnings.filterwarnings('ignore')
key = open('src/api.txt', 'r').read()

client = MongoClient()
db = client['meet-ups']
collection = db.groups
events_coll = db.events_two
attendees = db.attendees
categories = db.category_per_group

## Get number of groups in each category globally

In [8]:
from groups_per_category import get_count_of_category, get_ids_of_categories

ids, dict_categories = get_ids_of_categories()
groups_per_category = [get_count_of_category(dict_categories, category) for category in ids]

In [19]:
#Total number of groups

pprint(groups_per_category)
sum([val[1] for val in groups_per_category])

[[1, 7127, 'Arts & Culture'],
 [2, 51449, 'Career & Business'],
 [3, 2042, 'Cars & Motorcycles'],
 [4, 7473, 'Community & Environment'],
 [5, 4952, 'Dancing'],
 [6, 9451, 'Education & Learning'],
 [8, 1508, 'Fashion & Beauty'],
 [9, 8259, 'Fitness'],
 [10, 10135, 'Food & Drink'],
 [11, 7840, 'Games'],
 [12, 4375, 'LGBT'],
 [13, 5211, 'Movements & Politics'],
 [14, 27601, 'Health & Wellbeing'],
 [15, 4053, 'Hobbies & Crafts'],
 [16, 14182, 'Language & Ethnic Identity'],
 [17, 1488, 'Lifestyle'],
 [18, 4337, 'Book Clubs'],
 [20, 3508, 'Movies & Film'],
 [21, 7130, 'Music'],
 [22, 17416, 'New Age & Spirituality'],
 [23, 14577, 'Outdoors & Adventure'],
 [24, 407, 'Paranormal'],
 [25, 6974, 'Parents & Family'],
 [26, 2556, 'Pets & Animals'],
 [27, 3888, 'Photography'],
 [28, 6310, 'Religion & Beliefs'],
 [29, 1394, 'Sci-Fi & Fantasy'],
 [30, 2387, 'Singles'],
 [31, 23121, 'Socializing'],
 [32, 9925, 'Sports & Recreation'],
 [33, 7018, 'Support'],
 [34, 64501, 'Tech'],
 [36, 3508, 'Writing']

346103

I find 344,607 groups globally. Wow!

# Download all Meetup Groups

In [17]:
from get_groups_for_category import * 

for number in ids:
    get_groups_for_category(number, groups_per_category)

KeyboardInterrupt: 

# Visibilities

88% of the groups are public, 12% are public_limited, and 0.3% are members.

Public_limited allows you to see the events, but not who went. Members does not allow you to even see the events. 
Thus, these groups both need to be deleted.

In [118]:
def percent_each_visibility(collection):
    visibilities = collection.distinct('visibility')
    total_count = collection.count()
    percentages = [[option, round(collection.find({'visibility': option}).count() / total_count, 3) * 100] for option in visibilities]
    return percentages

#Remove these visibilities from meetups.

collection.delete_many({'visibility' : {'$in' : ['public_limited', 'members']}})

percent_each_visibility(collection)

[['public', 100.0]]

# Download the events for a particular group

In [140]:
req = requests.get('https://api.meetup.com/Florida-MUFON-UFO-Research-and-Investigations/events?\
                    key={}&status=past'.format(key))

print(req.headers['X-Total-Count'])
pprint(json.loads(req.text)[0])

168
{'created': 1351348658000,
 'date_in_series_pattern': False,
 'description': '<p>\xa0</p> <p>\xa0</p> <p>\xa0</p> <p>\xa0</p> <p>\xa0</p> '
                "<p>Our FIRST MUFON Meetup \xa0will be at <b>Mimi's "
                'Cafe</b></p> <p>at the Avenues in Viera, FL. \xa0From 5:30 pm '
                'to 8:30 pm.</p> <p>\xa0</p> <p>Please RSVP for this meeting '
                'and all future meetings.</p> <p>Feel free to bring a friend '
                'or two. See you there!</p> <p>\xa0</p> <p>Your Host for the '
                'evening will be</p> <p><b>MUFON</b> Assistant State Director '
                'Teri Lynge</p> ',
 'duration': 10800000,
 'group': {'country': 'us',
           'created': 1351346689000,
           'id': 5582872,
           'join_mode': 'open',
           'lat': 27.969999313354492,
           'localized_location': 'Palm Bay, FL',
           'lon': -80.6500015258789,
           'name': 'Florida MUFON UFO Research and Investigations',
           'reg

# Download all the events... ever


In [ ]:
from get_all_events import *

get_all_events(collection)


Something went wrong!
Inserted meetup-group-QceIsyDn into db.events!
Inserted Computer-Vision-and-Artificial-Intelligence-in-Agriculture into db.events!
Inserted Tantra-Lessons-in-Sacred-Sensuality into db.events!
Something went wrong!
Inserted Singapore-Skiing-Snowboarding into db.events!
Inserted Ecouter-parler-l-anglais-le-francais-et-le-chinois into db.events!
Inserted ProductTank-Cork into db.events!
Something went wrong!
Inserted Guitar-Circles-and-Acoustic-Jams into db.events!
Something went wrong!
Something went wrong!
Something went wrong!
Something went wrong!
Inserted Fayetteville-Hyperglots-and-Language-Learners-Group into db.events!
Inserted Bachata-and-Salsa-Dallas into db.events!
Inserted Rotterdam-Sound-Bath-Meetup into db.events!
Inserted Hilversum-Internet-of-things-IoT-SODAQ into db.events!
Inserted Hudson-Valley-Fiction-Writers-Workshop into db.events!


# Find the attendees for a particular event

In [135]:
req = requests.get('https://api.meetup.com/Florida-MUFON-UFO-Research-and-Investigations/events/88793112/rsvps?\
                    key={}'.format(key))

#Insert the attendees into their own database.
id_data = json.loads(req.text)

#get unique identifiers out
collection.distinct('member.id')

[7385260, 26274872, 71777132, 62218232, 16899141, 56597972, 12450926]

# Remove the groups for which events already done. 

In [2]:
f = open('new_groups.txt', 'r')
new_groups_done = f.read().replace('"', '').replace(",", "").splitlines()
f.close()

# get total list of groups from meet-ups.groups
# all_groups = set(collection.distinct('urlname'))

# # list of usernames in groups and not in groups_done
# groups_not_done = [group for group in all_groups if group not in groups_done]

# f= open("groups_not_done.txt","w+")
# f.write(str(groups_not_done))

# f = open('groups_not_done.txt', 'r')
# groups_not_done = f.read().replace("'", '').replace(" ", '').split(',')
# f.close()

# Metacategories

There are 33 categories, but only 24 meta-categories. Categories are merged into metacategories; for example, New Age & Spirituality, Paranormal, and Religion and Beliefs are all put into Beliefs. 

So metacategories is not likely to be a helpful category. 


In [141]:
meta_categories = [[element['id'],element['name'], element['category_ids']] for element in collection.distinct('meta_category')]

pprint(meta_categories)

[[132, 'Beliefs', [22, 24, 28]],
 [122, 'Arts', [1]],
 [212, 'Language & Culture', [16]],
 [272, 'Social', [31]],
 [583, 'Film', [20]],
 [162, 'Food & Drink', [10]],
 [292, 'Tech', [34]],
 [532, 'Hobbies & Crafts', [15]],
 [552, 'Movements', [4, 13]],
 [302, 'Health & Wellness', [14, 33]],
 [585, 'LGBTQ', [12]],
 [522, 'Career & Business', [2]],
 [582, 'Writing', [36]],
 [562, 'Learning', [6]],
 [512, 'Music', [21]],
 [242, 'Outdoors & Adventure', [3, 23]],
 [542, 'Dance', [5]],
 [182, 'Sci-Fi & Games', [11, 29]],
 [222, 'Book Clubs', [18]],
 [232, 'Family', [25]],
 [262, 'Photography', [27]],
 [282, 'Sports & Fitness', [9, 32]],
 [584, 'Fashion & Beauty', [8]],
 [252, 'Pets', [26]]]


# Create New MongoDB with categoryID for each group

In [ ]:
# Purpose: Need to get the different categories
# But the events db doesn't have category IDs. 
  
def make_mongo_db_with_category_id_for_group():  
    # create a new mongo collection that has category_id for every 10th group.
    for group in new_groups_done[2::10]:
        try:
            cat_id = db.groups.find_one({'urlname' : group})['category']['id']
            print(group, cat_id)
            categories.insert_one({'group' : group, 
                                   'category_id' : cat_id})
        except:
            print('Something went wrong')

make_mongo_db_with_category_id_for_group()


# Use category_per_group collection to get 100 groups per category

We have 191,651 groups.

Dict has event: number

1. Filter event by key > 50 - turn into list.
2. The function will take the urlname > find_all in events that match that urlname > for each of those events, do api call that gets attendees > insert that result into new mongo db. 

In [8]:
def pull_attendees_for_events_for_group(list_of_groups):
    '''
    For every group, we get all of their events. For every event, we get the list of attendees.
    We insert a document into the attendees collection with this information, as well as
    the date and time. 
    '''
    for group in list_of_groups:
        # If it's not already in there.
        if db.attendees.find_one({'urlname':group[1]}) is None:
            print('On Group:' + group[1])
            req = requests.get('https://api.meetup.com/{}/events?\
                                key={}&status=past&page=1&desc=true'.format(group[1], key))
            print(req.headers['X-RateLimit-Remaining'])
            if int(req.headers['X-Total-Count']) > 1:    
                events = events_coll.find({'group.urlname' : group[1]})
                print(events.count())
                for event in events:
                    # Only want events that people showed up to. The host doesn't count. 
                    if event['yes_rsvp_count'] > 1:
                        req = requests.get(
                            'https://api.meetup.com/{}/events/{}/rsvps?key={}&response=yes&only=member'\
                            .format(group[1], event['id'], key))
                        print(events.count())
                        time.sleep(0.25)
                        yeses = json.loads(req.text)
                        ids = [yes['member']['id'] for yes in yeses if yes['member']['event_context']['host'] != True]
                        attendees.insert_one({'group' : event['group']['id'],
                                              'urlname': group[1],
                                              'event' : event['id'],
                                              'date' : event['local_date'],
                                              'time' : event['local_time'],
                                             'attendees' : ids,
                                             'category_id' : group[0]})

pull_attendees_for_events_for_group(hundred_groups_per_cat[136:])

# Analysis: For every attendee, did you come back?

In [ ]:
def did_they_come_back():
    
    # Get list of urlnames in db.attendees
    urlnames = db.attendees.distinct('urlname')
    for urlname in urlnames:
        return db.attendees.find_one({'urlname':urlname})['category_id']
        events = db.attendees.find({'urlname':'The-Drawing-Circle-Valencia'})
        all_attendees_ever = [event['attendees'] for event in events]
        
        #turns list of lists into list.
        all_attendees_ever = [k for i in all_attendees_ever for k in i]
        
        #Counts number who have ever attended.
        all_attendees_ever_set = len(set(all_attendees_ever))
        
        #Counts their occurence, only keeps the returners.
        times_dict = Counter(all_attendees_ever)
        all_attendees_greater_than_1 = len([k for k, v in times_dict.items() if v > 1])
        percent_return = all_attendees_greater_than_1 / all_attendees_ever_set
        
        #Insert the data for each group into mongo
        db.returns_per_group.insert_one({'urlname' : urlname,
                                         'Percent_Return' : percent_return,
                                         'Category_ID': 'o'})
        
    
did_they_come_back()

# Next steps

Remove groups that have only had 1 event. 

Remove groups that have < n events.

How much do retention rates change over time? That would be beautiful to visualize.

Especially for Austin-Texas meetup group

## To figure out category popularity

* Get average number of attendees / event for every category.

* Total events per category (not very helpful)

* Average events / group per category. 